In [9]:
import sys

sys.path.append("../")

In [10]:
import requests
import time
import pandas as pd
import numpy as np
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from stats_arb.tests import adf_test, kpss_test, cal_half_life, pp_test
from datetime import datetime, timedelta
from ta.volatility import BollingerBands

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
plt.ioff()

In [11]:
API_BASE = 'https://fapi.binance.com/fapi/v1/'
# DATA_PATH = '/mnt/c/Users/vmodg/source/repos/BeanRepos/crypto-pair-trading/data/'
DATA_PATH = '/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/crypto/'
TIMEFRAME = '1d'
lookback = 365

TIMEFRAME = '15m'
lookback = 30

# TIMEFRAME = '5m'
# lookback = 20


In [12]:


LABELS = [
    'open_time',
    'open',
    'high',
    'low',
    'close',
    'volume',
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]

DROP_COLUMNS=[
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]


def get_batch(symbol, interval='1m', start_time=0, limit=400):
    """Use a GET request to retrieve a batch of candlesticks. Process the JSON into a pandas
    dataframe and return it. If not successful, return an empty dataframe.
    """

    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'limit': limit
    }
    try:
        # timeout should also be given as a parameter to the function
        response = requests.get(f'{API_BASE}klines', params, timeout=30)
    except requests.exceptions.ConnectionError:
        print('Connection error, Cooling down for 5 mins...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    except requests.exceptions.Timeout:
        print('Timeout, Cooling down for 5 min...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    if response.status_code == 200:
        return pd.DataFrame(response.json(), columns=LABELS)
    
    print(f'Got erroneous response back {symbol}: {response}. {response.text}')
    return pd.DataFrame([])


def get_candles(base, quote, start_date: datetime, end_date=None, interval='1m'):
    batches = []

    if end_date is None:
        end_date = datetime.utcnow()

    last_timestamp = int(start_date.timestamp()) * 1000
    # gather all candlesticks available, starting from the last timestamp loaded from disk or 0
    # stop if the timestamp that comes back from the api is the same as the last one
    previous_timestamp = None

    while previous_timestamp != last_timestamp:
        # stop if we reached data from today
        if datetime.fromtimestamp(last_timestamp / 1000) >= end_date:
            break

        previous_timestamp = last_timestamp

        new_batch = get_batch(
            symbol=base + quote,
            interval=interval,
            start_time=last_timestamp
        )

        # requesting candles from the future returns empty
        # also stop in case response code was not 200
        if new_batch.empty:
            break

        last_timestamp = new_batch['open_time'].max()

        # sometimes no new trades took place yet on date.today();
        # in this case the batch is nothing new
        if previous_timestamp == last_timestamp:
            break

        batches.append(new_batch)
        last_datetime = datetime.fromtimestamp(last_timestamp / 1000)

        covering_spaces = 20 * ' '
        print(datetime.now(), base, quote, interval, str(last_datetime) + covering_spaces, end='\r', flush=True)

    if len(batches) > 0:
        # write clean version of csv to parquet
        df = pd.concat(batches, ignore_index=True)
        df.drop(columns=DROP_COLUMNS, inplace=True)
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        df.set_index('open_time', inplace=True)
        df = df[~df.index.duplicated(keep='first')]
        return df



In [13]:
# import requests

# url = "https://fapi.binance.com/fapi/v1/exchangeInfo"

# payload={}
# headers = {}

# response = requests.request("GET", url, headers=headers, data=payload).json()

# symbols = [s['symbol'] for s in response['symbols'] if s['contractType'] == 'PERPETUAL' and s['quoteAsset'] == 'USDT']
# # symbols[100:]
# symbols = symbols[:50]
# len(symbols)


In [14]:
# symbols = pd.read_csv(f'/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/symbols.csv')['symbol'].values.tolist()
symbols = ['BTCUSDT',
'ETHUSDT',
'BCHUSDT',
'XRPUSDT',
'LTCUSDT',
'ETCUSDT',
'LINKUSDT',
'XLMUSDT',
'ADAUSDT',
'BNBUSDT',
'ATOMUSDT',
'ALGOUSDT',
'DOTUSDT',
'SOLUSDT',
'AVAXUSDT',
'MATICUSDT',
'XMRUSDT',
'NEARUSDT',
'AVAXUSDT',
'ATOMUSDT',
'MANAUSDT',
'UNIUSDT',
'BCHUSDT']
symbols = [s.replace('USDT', '') for s in symbols]

data = []
start_time = datetime.utcnow() - timedelta(days=lookback)

for symbol in symbols:
    df = pd.read_csv(f'{DATA_PATH}/{TIMEFRAME}/{symbol}-USDT.csv', parse_dates=['open_time'], index_col=['open_time'])
    df = df[df.index >= start_time].copy()
    # print('start time', df.index[-1].to_pydatetime())

    df1 = get_candles(
        base=symbol, 
        quote='USDT', 
        start_date=df.index[-1].to_pydatetime(),
        # end_date=datetime.now() - timedelta(days=15),
        interval=TIMEFRAME
    )
    if df1 is not None:
        df = pd.concat([df, df1])
        df = df[~df.index.duplicated(keep='first')]

    df.rename(columns={'close': symbol}, inplace=True)
    # the data is too long, just limit to recent period
    log = np.log(df[symbol].astype(np.float32))
    data.append(log)

df = pd.concat(data, axis=1)
df = df.dropna(axis=1, how='all')
# df.dropna(inplace=True, how='any')

# our of sample
# df = df[df.index <= datetime.now() - timedelta(days=15)].copy()

df.tail()

,BTC,ETH,BCH,XRP,LTC,ETC,LINK,XLM,ADA,BNB,...,SOL,AVAX,MATIC,XMR,NEAR,AVAX,ATOM,MANA,UNI,BCH
open_time,,,,,,,,,,,,,,,,,,,,,
2023-04-15 01:30:00,10.321428,7.641699,4.877104,-0.655081,4.556610,3.100092,2.034836,-2.242525,-0.835863,5.793562,...,3.192244,2.951153,0.140892,5.085248,0.786182,2.951153,2.496506,-0.454760,1.834979,4.877104
2023-04-15 01:45:00,10.320740,7.640652,4.876570,-0.657973,4.558184,3.098018,2.034444,-2.242431,-0.837942,5.792800,...,3.192532,2.951362,0.141934,5.082770,0.785726,2.951362,2.497082,-0.453815,1.835776,4.876570
2023-04-15 02:00:00,10.321556,7.641843,4.876723,-0.656044,4.561427,3.099281,2.034575,-2.240832,-0.837480,5.794019,...,3.193394,2.953555,0.142367,5.083142,0.785726,2.953555,2.496423,-0.452399,1.835457,4.876723
2023-04-15 02:15:00,10.321487,7.642783,4.875579,-0.654696,4.562993,3.099326,2.036273,-2.240550,-0.836325,5.794537,...,3.193312,2.952981,0.142974,5.084567,0.787093,2.952981,2.496506,-0.453501,1.835298,4.875579
2023-04-15 02:30:00,10.321487,7.643574,4.874281,-0.654311,4.561845,3.099462,2.036012,-2.240362,-0.835863,5.795358,...,3.190888,2.950578,0.142801,5.086114,0.784814,2.950578,2.494857,-0.454918,1.831941,4.874281


In [15]:
p = 1
COINTEGRATION_CONFIDENCE_LEVEL = 90

# the 90%, 95%, and 99% confidence levels for the trace statistic and maximum 
# eigenvalue statistic are stored in the first, second, and third column of 
# cvt and cvm, respectively
confidence_level_cols = {
    90: 0,
    95: 1,
    99: 2
}
confidence_level_col = confidence_level_cols[COINTEGRATION_CONFIDENCE_LEVEL]


def test_johansen(symbol_pairs):
    df_t = df[symbol_pairs].copy()
    df_t.dropna(inplace=True)

    # The second and third parameters indicate constant term, with a lag of 1. 
    result = coint_johansen(df_t, 0, p)

    trace_crit_value = result.cvt[:, confidence_level_col]
    eigen_crit_value = result.cvm[:, confidence_level_col]
#     print("trace_crit_value",trace_crit_value)
#     print("eigen_crit_value",eigen_crit_value)
#     print("lr1",result.lr1)
#     print("lr2",result.lr2)

    # The trace statistic and maximum eigenvalue statistic are stored in lr1 and lr2;
    # see if they exceeded the confidence threshold
    if np.all(result.lr1 >= trace_crit_value) and np.all(result.lr2 >= eigen_crit_value):
        # print(f"{symbol_pairs} are cointegrated")
        # The first i.e. leftmost column of eigenvectors matrix, result.evec, contains the best weights.
        v1= result.evec[:,0:1]
        hr=v1/-v1[1] #to get the hedge ratio divide the best_eigenvector by the negative of the second component of best_eigenvector
        #the regression will be: close of symbList[1] = hr[0]*close of symbList[0] + error
        #where the beta of the regression is hr[0], also known as the hedge ratio, and
        #the error of the regression is the mean reverting residual signal that you need to predict, it is also known as the "spread"
        #the spread = close of symbList[1] - hr[0]*close of symbList[0] or alternatively (the same thing):
        #do a regression with close of symbList[0] as x and lose of symbList[1] as y, and take the residuals of the regression to be the spread.
        coint_pair = dict(hedge_ratio=v1[:, 0])
        for i, s in enumerate(symbol_pairs):
            coint_pair[f'sid_{i+1}'] = s

        cointegrating_pairs.append(coint_pair)



In [16]:
import traceback


def calculate_spread(df, coint_df, selected_row, hedge_ratio, nb_symbols=2):
    spread = None
    for i in range(nb_symbols):
        if spread is None:
            spread = df[coint_df[f'sid_{i + 1}'].iloc[selected_row]] * hedge_ratio[i]
        else:
            spread += df[coint_df[f'sid_{i + 1}'].iloc[selected_row]] * hedge_ratio[i]
    
    spread.dropna(inplace=True)
    return spread


critical_val = 0.05

def find_stationary_portfolio(coint_df):
    data = []
    for i, _ in coint_df.iterrows():
        try:
            hedge_ratio = coint_df.iloc[i]['hedge_ratio']
            _df = df.copy()
            spread = calculate_spread(_df, coint_df, i, hedge_ratio)
            p_val = adf_test(spread, verbose=False)
            if p_val < critical_val:
                half_life = cal_half_life(spread)
                pairs_name = coint_df[[col for col in coint_df.columns if 'sid' in col]].iloc[i].values
                # print(i, pairs_name, 'is stationary with half life', half_life)
                # print(' ')
                data.append({
                    'i': i,
                    'pairs': pairs_name,
                    'half_life': half_life
                })
        except:
            print(coint_df.iloc[i])
            # traceback.print_exc()
    
    return pd.DataFrame(data)


# find_stationary_portfolio(coint_df)

In [17]:

import itertools as it

nb_symbols = 2
cointegrating_pairs = []

#get symbol pairs
pairs = list(it.combinations(symbols, nb_symbols))

for pair in pairs:
    try:
        test_johansen(list(pair))
    except Exception:
        pass

coint_df = pd.DataFrame(cointegrating_pairs)
coint_df['i'] = coint_df.index
coint_df.head()

,hedge_ratio,sid_1,sid_2,i
0,"[29.706259686410085, -43.05253242708679]",BTC,ETC,0
1,"[8.35816299152634, -38.948625249938125]",BTC,LINK,1
2,"[31.0867499698863, -7.104355202097104]",BTC,SOL,2
3,"[3.538289902689712, 31.153764239023594]",BTC,MATIC,3
4,"[10.674051940320068, -49.217698779471114]",BTC,XMR,4


In [18]:
coint_df.to_csv(f'coint_df_{TIMEFRAME}.csv')


# Result

In [19]:
import warnings

warnings.filterwarnings("ignore")

stationary_df = find_stationary_portfolio(coint_df)
stationary_df = pd.merge(stationary_df, coint_df, on='i')
stationary_df.dropna(inplace=True)
stationary_df['i'] = stationary_df.i.astype('int64')
stationary_df['half_life'] = stationary_df.half_life.astype('int64')
if len(stationary_df) > 0:
    stationary_df.sort_values(by=['half_life'], inplace=True)
# stationary_df


In [20]:
stationary_df.to_csv(f'stationary_df_{TIMEFRAME}.csv', index=False)
stationary_df

,i,pairs,half_life,hedge_ratio,sid_1,sid_2
11,12,"[LTC, XMR]",40,"[26.261025275804247, -70.2174602857272]",LTC,XMR
31,34,"[MATIC, XMR]",49,"[22.88928805493115, 35.38803655742525]",MATIC,XMR
36,39,"[XMR, UNI]",53,"[37.725794974868165, 18.231298933440883]",XMR,UNI
38,41,"[MANA, UNI]",61,"[49.52048184859548, -34.103436828046114]",MANA,UNI
37,40,"[NEAR, MANA]",61,"[28.550842462321206, -50.969655682090156]",NEAR,MANA
26,27,"[BNB, XMR]",62,"[20.623325475139183, 36.700436843602176]",BNB,XMR
8,9,"[XRP, XMR]",63,"[4.838184045552107, -48.98621272732247]",XRP,XMR
35,38,"[XMR, MANA]",65,"[27.97782190926404, 18.064316009317558]",XMR,MANA
3,4,"[BTC, XMR]",69,"[10.674051940320068, -49.217698779471114]",BTC,XMR
10,11,"[LTC, MATIC]",69,"[14.583285941040497, 26.825044137535897]",LTC,MATIC
